In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json

In [2]:
business = pd.read_pickle("c:/Users/leand/Desktop/Proyecto Final/Yelp/Datasets Yelp/business.pkl")
tip = pd.read_json("c:/Users/leand/Desktop/Proyecto Final/Yelp/Datasets Yelp/tip.json", lines=True)
user = pd.read_parquet("c:/Users/leand/Desktop/Proyecto Final/Yelp/Datasets Yelp/user.parquet")

# BUSINESS

In [3]:
# Eliminar las columnas duplicadas, conservando la primera instancia de cada columna
business = business.loc[:,~business.columns.duplicated(keep='first')]

In [4]:
# dejamos solo la data del estado de FL (Florida)
business_fl = business[business['state'] == 'FL']

In [5]:
business_fl.info()

<class 'pandas.core.frame.DataFrame'>
Index: 26329 entries, 10 to 150320
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   business_id   26329 non-null  object
 1   name          26329 non-null  object
 2   address       26329 non-null  object
 3   city          26329 non-null  object
 4   state         26329 non-null  object
 5   postal_code   26329 non-null  object
 6   latitude      26329 non-null  object
 7   longitude     26329 non-null  object
 8   stars         26329 non-null  object
 9   review_count  26329 non-null  object
 10  is_open       26329 non-null  object
 11  attributes    23883 non-null  object
 12  categories    26311 non-null  object
 13  hours         22221 non-null  object
dtypes: object(14)
memory usage: 3.0+ MB


In [6]:
business_fl.loc[:, 'categories'] = business_fl['categories'].fillna('')

In [7]:
business_fl = business_fl.loc[:, ~business_fl.columns.isin(['postal_code', 'is_open', 'hours'])]


In [8]:
business_fl.head(2)

,business_id,name,address,city,state,latitude,longitude,stars,review_count,attributes,categories
10,UJsufbvfyfONHeWdvAHKjA,Marshalls,21705 Village Lakes Sc Dr,Land O' Lakes,FL,28.190459,-82.45738,3.5,6,"{'RestaurantsPriceRange2': '2', 'BikeParking':...","Department Stores, Shopping, Fashion"
13,jaxMSoInw8Poo3XeMJt8lQ,Adams Dental,15 N Missouri Ave,Clearwater,FL,27.966235,-82.787412,5.0,10,{'ByAppointmentOnly': 'True'},"General Dentistry, Dentists, Health & Medical,..."


In [9]:
business_fl.dropna(subset=['categories'], inplace=True)

In [10]:
categorias_deseadas = [
    'Restaurants', 'Food', 'Bubble Tea', 'Coffee & Tea', 'Bakeries', 'Brewpubs', 'Breweries', 'Burgers', 'Fast Food',
    'Sandwiches', 'Ice Cream & Frozen Yogurt', 'Diners', 'Breakfast & Brunch', 'Sushi Bars',
    'Cafes', 'Wine Bars', 'Steakhouses', 'Asian Fusion', 'Hot Dogs', 'Seafood', 'Cocktail Bars', 'Pizza',
    'Chicken Wings', 'Salad', 'Soup', 'Juice Bars & Smoothies', 'American (New)', 'Sports Bars', 'Gastropubs',
    'Beer Bar', 'Cajun/Creole'
]

# Filtrar el DataFrame
business_fl = business_fl[business_fl['categories'].apply(lambda x: any(cat in x for cat in categorias_deseadas))]

# Reiniciar los índices del nuevo DataFrame
business_fl = business_fl.reset_index(drop=True)

In [11]:
business_fl.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11413 entries, 0 to 11412
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   business_id   11413 non-null  object
 1   name          11413 non-null  object
 2   address       11413 non-null  object
 3   city          11413 non-null  object
 4   state         11413 non-null  object
 5   latitude      11413 non-null  object
 6   longitude     11413 non-null  object
 7   stars         11413 non-null  object
 8   review_count  11413 non-null  object
 9   attributes    11277 non-null  object
 10  categories    11413 non-null  object
dtypes: object(11)
memory usage: 980.9+ KB


In [12]:
business_fl.head(2)

,business_id,name,address,city,state,latitude,longitude,stars,review_count,attributes,categories
0,0bPLkL0QhhPO5kt1_EXmNQ,Zio's Italian Market,2575 E Bay Dr,Largo,FL,27.916116,-82.760461,4.5,100,"{'OutdoorSeating': 'False', 'RestaurantsGoodFo...","Food, Delis, Italian, Bakeries, Restaurants"
1,cVBxfMC4lp3DnocjYA3FHQ,Paws The Cat Cafe,10588 109 Street,Edmonton,FL,53.549633,-113.50878,5.0,20,"{'RestaurantsAttire': ''casual'', 'Restaurants...","Coffee & Tea, Cafes, Pets, Restaurants, Pet Ad..."


In [13]:
# Suponiendo que 'business_fl' es el DataFrame
business_id_list = business_fl['business_id'].tolist()

# TIP

In [14]:
tip.head()

,user_id,business_id,text,date,compliment_count
0,AGNUgVwnZUey3gcPCJ76iw,3uLgwr0qeCNMjKenHJwPGQ,Avengers time with the ladies.,2012-05-18 02:17:21,0
1,NBN4MgHP9D3cw--SnauTkA,QoezRbYQncpRqyrLH6Iqjg,They have lots of good deserts and tasty cuban...,2013-02-05 18:35:10,0
2,-copOvldyKh1qr-vzkDEvw,MYoRNLb5chwjQe3c_k37Gg,It's open even when you think it isn't,2013-08-18 00:56:08,0
3,FjMQVZjSqY8syIO-53KFKw,hV-bABTK-glh5wj31ps_Jw,Very decent fried chicken,2017-06-27 23:05:38,0
4,ld0AperBXk1h6UbqmM80zw,_uN0OudeJ3Zl_tf6nxg5ww,Appetizers.. platter special for lunch,2012-10-06 19:43:09,0


In [15]:
# Suponiendo que 'tip' es tu DataFrame y 'business_id_list' es la lista creada anteriormente
tip_fl = tip[tip['business_id'].isin(business_id_list)]

In [16]:
print(len(business_id_list))

11413


In [17]:
tip.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 908915 entries, 0 to 908914
Data columns (total 5 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   user_id           908915 non-null  object        
 1   business_id       908915 non-null  object        
 2   text              908915 non-null  object        
 3   date              908915 non-null  datetime64[ns]
 4   compliment_count  908915 non-null  int64         
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 34.7+ MB


In [18]:
tip_fl.head()

,user_id,business_id,text,date,compliment_count
1,NBN4MgHP9D3cw--SnauTkA,QoezRbYQncpRqyrLH6Iqjg,They have lots of good deserts and tasty cuban...,2013-02-05 18:35:10,0
15,j2sEA3hiUcwHfq9Ml6M47g,EXYbKA1tocvOK_1tXxZXLA,Don't go for dinner. They close at 6. Really Y...,2011-10-13 03:15:15,0
17,kjFgyrCvmVVGSlgWzRXILw,ReX09lhufLTAx19krkltDA,Got the grilled chicken entree. It was delicio...,2013-06-10 20:18:41,0
31,fjpoZaeM1QRueYLl9FSLiQ,8YJtf5v76SIz-4udyF4YmA,This place had great coffee but the owner sold...,2017-09-23 16:08:24,0
46,evVjF1TOaNwuUK7oY2k53w,5yjHCLzwHgjKNNHgAaUC8Q,Jimmy the manager does what he is best at agai...,2012-03-28 23:27:41,0


# USER

In [27]:
user.head(2)

,user_id,name,review_count,useful,funny,cool,fans,average_stars,compliment_count
0,qVc8ODYU5SZjKXVBgXdI7w,Walker,585,7217,1259,5994,267,3.91,2873
1,j14WgRoU_-2ZE1aw1dXrJg,Daniel,4333,43091,13066,27281,3138,3.74,20631


In [20]:
user.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2105597 entries, 0 to 2105596
Data columns (total 22 columns):
 #   Column              Dtype  
---  ------              -----  
 0   user_id             object 
 1   name                object 
 2   review_count        int64  
 3   yelping_since       object 
 4   useful              int64  
 5   funny               int64  
 6   cool                int64  
 7   elite               object 
 8   friends             object 
 9   fans                int64  
 10  average_stars       float64
 11  compliment_hot      int64  
 12  compliment_more     int64  
 13  compliment_profile  int64  
 14  compliment_cute     int64  
 15  compliment_list     int64  
 16  compliment_note     int64  
 17  compliment_plain    int64  
 18  compliment_cool     int64  
 19  compliment_funny    int64  
 20  compliment_writer   int64  
 21  compliment_photos   int64  
dtypes: float64(1), int64(16), object(5)
memory usage: 353.4+ MB


In [21]:
# Suponiendo que 'user' es tu DataFrame
columns_to_sum = ['compliment_hot', 'compliment_more', 'compliment_profile', 'compliment_cute',
                   'compliment_list', 'compliment_note', 'compliment_plain', 'compliment_cool',
                   'compliment_funny', 'compliment_writer', 'compliment_photos']

# Crear la nueva columna 'compliment_count' sumando las columnas seleccionadas
user['compliment_count'] = user[columns_to_sum].sum(axis=1)

In [22]:
# Eliminar las columnas originales
user.drop(columns=columns_to_sum, inplace=True)

In [23]:
user.drop(columns='friends', inplace=True)
user.drop(columns='elite', inplace=True)
user.drop(columns='yelping_since', inplace=True)

# REVIEW

In [24]:
datos = []

with open('c:/Users/leand/Desktop/Proyecto Final/Yelp/Datasets Yelp/review.json', encoding="utf-8") as archivo:
    for linea in archivo:
        datos.append(json.loads(linea))

reviews = pd.DataFrame(datos)

In [25]:
# Suponiendo que 'tip' es tu DataFrame y 'business_id_list' es la lista creada anteriormente
reviews_fl = reviews[reviews['business_id'].isin(business_id_list)]

In [30]:
reviews_fl.info()

<class 'pandas.core.frame.DataFrame'>
Index: 895976 entries, 7 to 6990279
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   review_id    895976 non-null  object 
 1   user_id      895976 non-null  object 
 2   business_id  895976 non-null  object 
 3   stars        895976 non-null  float64
 4   useful       895976 non-null  int64  
 5   funny        895976 non-null  int64  
 6   cool         895976 non-null  int64  
 7   text         895976 non-null  object 
 8   date         895976 non-null  object 
dtypes: float64(1), int64(3), object(5)
memory usage: 68.4+ MB


In [28]:
reviews_fl.head(2)

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
7,_ZeMknuYdlQcUqng_Im3yg,yfFzsLmaWF2d4Sr0UNbBgg,LHSTtnW3YHCeUkRDGyJOyw,5.0,2,0,0,Amazingly amazing wings and homemade bleu chee...,2015-08-07 02:29:16
9,pUycOfUwM8vqX7KjRRhUEA,59MxRhNVhU9MYndMkz0wtw,gebiRewfieSdtt17PTW6Zg,3.0,0,0,0,Had a party of 6 here for hibachi. Our waitres...,2016-07-25 07:31:06


In [31]:
'''
business_fl.to_json('yelp_business_fl.json', orient='records')
tip_fl.to_json('yelp_tip_fl.json', orient='records')
user.to_json('yelp_user.json', orient='records')
reviews_fl.to_json('yelp_reviews_fl.json', orient='records')
'''